### Simple Gen AI APP udsing langchain

In [1]:
import os 
from rich import print 
from dotenv import load_dotenv 
load_dotenv(dotenv_path="../.env")

True

In [3]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "BASIC_GENAI_APP"
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

#### DATA Ingestion --> from the website we need to scrape the data

In [5]:
from langchain_community.document_loaders import WebBaseLoader

In [6]:
loader=WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
loader

In [7]:
docs=loader.load()

In [10]:
print(docs)

[
    Document(
        metadata={
            'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend',
            'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith',
            'description': 'Before diving into this content, it might be helpful to read the following:',
            'language': 'en'
        },
        page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main 
contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.API 
ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment 
(LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual 
GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication 
methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing
spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended ReadingBefore diving into this 
content, it might be helpful to read the following:\nData Retention Conceptual Docs\nUsage Limiting Conceptual 
Docs\n\nnoteSome of the features mentioned in this guide are not currently available in Enterprise plan due to 
its\ncustom nature of billing. If you are on Enterprise plan and have questions about cost optimization,\nplease 
reach out to your sales rep or support@langchain.dev.\nThis tutorial walks through optimizing your spend on 
LangSmith. In it, we will learn how to optimize existing spend\nand prevent future overspend on a realistic 
real-world example. We will use an existing LangSmith organization with high usage.\nConcepts can be transferred to
your own organization.\nProblem Setup\u200b\nIn this tutorial, we take an existing organization that has three 
workspaces, one for each deployment stage\n(Dev, Staging, and Prod):\n\nUnderstand your current usage\u200b\nThe 
first step of any optimization process is to understand current usage. LangSmith gives two ways to do this: Usage 
Graph\nand Invoices.\nUsage Graph\u200b\nThe usage graph lets us examine how much of each usage based pricing 
metric we have consumed lately. It does not directly show\nspend (which we will see later on our draft 
invoice).\nWe can navigate to the Usage Graph under Settings -> Usage and Billing -> Usage Graph.\n\nWe see in the 
graph above that there are two usage metrics that LangSmith charges for:\n\nLangSmith Traces (Base 
Charge)\nLangSmith Traces (Extended Data Retention Upgrades).\n\nThe first metric tracks all traces that you send 
to LangSmith. The second tracks all traces that also have our Extended 400 Day Data Retention.\nFor more details, 
see our data retention conceptual docs. Notice that these graphs look\nidentical, which will come into play later 
in the tutorial.\nLangSmith Traces usage is measured per workspace, because workspaces often represent development 
environments (as in our example),\nor teams within an organization. As a LangSmith administrator, we want to 
understand spend granularly per each of these units. In\nthis case where we just want to cut spend, we can focus on
the environment responsible for the majority of costs first for the greatest savings.\nInvoices\u200b\nWe 
understand what usage looks like in terms of traces, but we now need to translate that into spend. To do so,\nwe 
head to the Invoices tab. The first invoice that will appear on screen is a draft of your current 
month\'s\ninvoice, which shows your running spend thus far this month.\n\nnoteLangSmith\'s Usage Graph and Invoice 
use the term tenant_id to refer to a workspace ID. They are interchangeable.\nIn the above GIF, we see that the 
charges for LangSmith Traces are broken up by "tenant_id" (i.e. Workspace ID), meaning we can track tracing 
spend\non each of our workspaces. In the first few days of June, the vast majority of the to

In [9]:
len(docs)

1

#### Work flow

load Data--> Docs--> chunks --(embedding model)--> vectors -->vector Store DB

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

chunks=text_splitter.split_documents(docs)


In [15]:
print(chunks[:2]) # first two chunks

[
    Document(
        metadata={
            'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend',
            'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith',
            'description': 'Before diving into this content, it might be helpful to read the following:',
            'language': 'en'
        },
        page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur
new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.API 
ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment 
(LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual 
GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication 
methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing
spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended ReadingBefore diving into this 
content, it might be helpful to read the following:\nData Retention Conceptual Docs\nUsage Limiting Conceptual 
Docs'
    ),
    Document(
        metadata={
            'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend',
            'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith',
            'description': 'Before diving into this content, it might be helpful to read the following:',
            'language': 'en'
        },
        page_content='noteSome of the features mentioned in this guide are not currently available in Enterprise 
plan due to its\ncustom nature of billing. If you are on Enterprise plan and have questions about cost 
optimization,\nplease reach out to your sales rep or support@langchain.dev.\nThis tutorial walks through optimizing
your spend on LangSmith. In it, we will learn how to optimize existing spend\nand prevent future overspend on a 
realistic real-world example. We will use an existing LangSmith organization with high usage.\nConcepts can be 
transferred to your own organization.\nProblem Setup\u200b\nIn this tutorial, we take an existing organization that
has three workspaces, one for each deployment stage\n(Dev, Staging, and Prod):'
    )
]

In [17]:
len(chunks) # we have total 17 chunks

17

#### Embeddings

In [18]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()


In [21]:
from langchain_community.vectorstores import FAISS 

vector_db=FAISS.from_documents(chunks,embedding=embeddings)

In [23]:
vector_db

In [24]:
query="LangSmith has tow usage limits:total traces and extended"

In [25]:
results=vector_db.similarity_search(query)

In [28]:
print(results[0].page_content)

Optimization 2: limit usage​
In the previous section, we managed data retention settings to optimize existing spend. In this section, we will
use usage limits to prevent future overspend.
LangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics 
we've
been tracking on our usage graph. We can use these in tandem to have granular control over spend.
To set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the
bottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along
with a cost estimate:

Lets start by setting limits on our production usage, since that is where the majority of spend comes from.
Setting a good total traces limit​
Picking the right "total traces" limit depends on the expected load of traces that you will send to LangSmith. You 
should
clearly think about your assumptions before setting a limit.
For example:

In [30]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
llm_groq=ChatGroq(model="openai/gpt-oss-20b")  
llm_openai=ChatOpenAI(model="gpt-5")

#### Retrieval chain , Document chain

In [31]:
# create a chain for passing a list of documents to a model 
from langchain.chains.combine_documents import create_stuff_documents_chain 
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
"""
    Answer the following question based only on the provided context:
    <context>
        {context}
    </context>


"""
)

document_chain=create_stuff_documents_chain(llm_openai,prompt)



In [ ]:
document_chain # chain --> prompt | llm | output parser

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n        {context}\n    </context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x138cc16c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x138cc0b80>, root_client=<openai.OpenAI object at 0x138cc1f00>, root_async_client=<openai.AsyncOpenAI object at 0x138cc2230>, model_name='gpt-5', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_n

In [ ]:
from langchain_core.documents import Document

results_chain=document_chain.invoke(

    {
        "input":"LangSmith has tow usage limits:total traces and extended",
        "context":[Document(page_content='''LangSmith has tow usage limits:total traces and extended.
                            These correspond to the two metrics we've been tracking on our usage graph. ''')]
    }
)
print(results_chain)

LangSmith has two usage limits: total traces and extended, which correspond to the two metrics tracked on the usage
graph.

However, we want the documents to first come from the retriever we just set up.That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

### Input --> Retriever ----> vectorStoreDB

we will convert vector DB into retriever , so no need to use similarity search and easy to use in chain 

In [37]:
# create a retriever 

retriever=vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11f619540>, search_kwargs={})

In [48]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11f619540>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n        {context}\n    </context>\n\n\n'), additional_kwargs={})])
 

In [43]:
## Get the response from the llm 
respomse_RC=retrieval_chain.invoke({"input":"LangSmith has tow usage limits:total traces and extended"})


In [51]:
print(respomse_RC['answer'])

I don’t see a specific question in your message. Could you please provide the question you want answered based on 
the context?

In [53]:
print(respomse_RC)

{
    'input': 'LangSmith has tow usage limits:total traces and extended',
    'context': [
        Document(
            id='c9d828b1-2bac-4e1d-8791-7a897a1e1a52',
            metadata={
                'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend',
                'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith',
                'description': 'Before diving into this content, it might be helpful to read the following:',
                'language': 'en'
            },
            page_content='Optimization 2: limit usage\u200b\nIn the previous section, we managed data retention 
settings to optimize existing spend. In this section, we will\nuse usage limits to prevent future 
overspend.\nLangSmith has two usage limits: total traces and extended retention traces. These correspond to the two
metrics we\'ve\nbeen tracking on our usage graph. We can use these in tandem to have granular control over 
spend.\nTo set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table 
at the\nbottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear,
along\nwith a cost estimate:\n\nLets start by setting limits on our production usage, since that is where the 
majority of spend comes from.\nSetting a good total traces limit\u200b\nPicking the right "total traces" limit 
depends on the expected load of traces that you will send to LangSmith. You should\nclearly think about your 
assumptions before setting a limit.\nFor example:'
        ),
        Document(
            id='68c9a66b-f5e2-4cfb-ad24-7d76cf13ee14',
            metadata={
                'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend',
                'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith',
                'description': 'Before diving into this content, it might be helpful to read the following:',
                'language': 'en'
            },
            page_content="The first metric tracks all traces that you send to LangSmith. The second tracks all 
traces that also have our Extended 400 Day Data Retention.\nFor more details, see our data retention conceptual 
docs. Notice that these graphs look\nidentical, which will come into play later in the tutorial.\nLangSmith Traces 
usage is measured per workspace, because workspaces often represent development environments (as in our 
example),\nor teams within an organization. As a LangSmith administrator, we want to understand spend granularly 
per each of these units. In\nthis case where we just want to cut spend, we can focus on the environment responsible
for the majority of costs first for the greatest savings.\nInvoices\u200b\nWe understand what usage looks like in 
terms of traces, but we now need to translate that into spend. To do so,\nwe head to the Invoices tab. The first 
invoice that will appear on screen is a draft of your current month's\ninvoice, which shows your running spend thus
far this month."
        ),
        Document(
            id='565fe570-b785-4e4c-866e-b4064c39feca',
            metadata={
                'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend',
                'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith',
                'description': 'Before diving into this content, it might be helpful to read the following:',
                'language': 'en'
            },
            page_content='Understand your current usage\u200b\nThe first step of any optimization process is to 
understand current usage. LangSmith gives two ways to do this: Usage Graph\nand Invoices.\nUsage Graph\u200b\nThe 
usage graph lets us examine how much of each usage based pricing metric we have consumed lately. It does not 
directly show\nspend (which we will see later on our draft invoice).\nWe can navigate to the Usage Graph under 
Settings -> Usage and Billing -> Usage Gra